In [1]:
import math
import numpy as np

In [2]:
# Funções úteis
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sigmoidArray(array):
    return 1 / (1 + np.exp(-array))

def lostFunction(realValue, calculatedValue):
    return -((realValue * math.log10(calculatedValue)) + ((1-realValue) * math.log10(1-calculatedValue)))

def lostFunctionArray(referenceVector, calculatedVector):
    return -(np.dot(referenceVector, np.log10(calculatedVector).T) + np.dot((1-referenceVector), np.log10(1-calculatedVector).T))

In [3]:
# Variáveis
m = int(6e2)                                            # Quantidade total de dados
nx = int(2e2)                                           # Número de features por entrada

# Geração aleatória dos dados para a simulação da aplicação da regressão logística com descida de gradiente
X = np.random.rand(nx, m)                         # Entrada, podem ser imagens, sons, etc. Cada coluna representa 1 conjunto de dados de entrada.     ##Formato: (nx, m)
Y = np.random.randint(0, 2, m).reshape(1,m)                # Saída(Entrada), pode ser uma classificação de animais, estilo musical, etc.              ##Formato: (1, m)
TrainingSet = np.array([X, Y]).reshape(2,1)       # Conjunto de todos os pares ordenados (entrada, saída) dos exemplos de treinamento.                ##Formato: (2, 1)

# Inicialização dos parâmetros de ajuste da IA, parâmetros que sofrerão mudanças para tentar prever o resultado da saída
w = np.zeros((nx, 1))                             # Parâmetros de ajuste dos dados de treinamento (iniciado em 0)                                     ##Formato: (nx,1)
b = np.zeros((1,1))                               # Parâmetro de ajuste dos dados de treinamento (iniciado em 0)                                      ##Formato: (1,1)

print("Novos dados carregados com sucesso!\n")

Novos dados carregados com sucesso!



C:\Users\rapha\AppData\Local\Temp/ipykernel_6492/399605692.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  TrainingSet = np.array([X, Y]).reshape(2,1)       # Conjunto de todos os pares ordenados (entrada, saída) dos exemplos de treinamento.                ##Formato: (2, 1)


In [4]:
print(f"TrainingSet (conjunto de dados), formato {TrainingSet.shape}:\n{TrainingSet}"
      f"\n\nX (Entrada), formato {X.shape}:\n{X}"

      f"\n\nY (Saída), formato {Y.shape}:\n{Y}"
      f"\n\nw (peso por entrada), formato {w.shape}:\n{w}"
      f"\n\nb (peso geral), formato {b.shape}:\n{b}")

TrainingSet (conjunto de dados), formato (2, 1):
[[array([[0.70415561, 0.21193451, 0.04865998, ..., 0.82367761, 0.62298973,
          0.10936805],
         [0.62227349, 0.38107434, 0.85550264, ..., 0.57153429, 0.76011557,
          0.23286273],
         [0.95207365, 0.8176934 , 0.71338929, ..., 0.05637039, 0.8845594 ,
          0.42999502],
         ...,
         [0.02418407, 0.98305601, 0.71465278, ..., 0.07614007, 0.87704163,
          0.62222519],
         [0.48196848, 0.00269318, 0.44023525, ..., 0.20628209, 0.52764479,
          0.71686371],
         [0.79344885, 0.12396114, 0.2247213 , ..., 0.94729472, 0.55155864,
          0.34265503]])                                                   ]
 [array([[0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
          1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0,
          1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0,
          0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0

In [5]:
def accuracyCalc():
    """
    Testa a regressão logística atual comparando os resultados obtidos com os corretos.

    :returns: array = [Valores predizidos - formato(1, m), precisão das predições]
    """
    prediction = np.zeros(m).reshape(1, m)
    rights = 0

    # Preenche o vetor de predição e calcula quantas vezes houve sucesso em prever o valor correto
    Z = np.dot(w.T, X) + b
    A = sigmoidArray(Z)
    prediction = A

    for i in range(m):
        if (prediction[0][i] > 0.5) and (Y[0][i] > 0.5):
            rights += 1
        if (prediction[0][i] <= 0.5) and (Y[0][i] <= 0.5):
            rights += 1

    return [prediction, rights / m]

In [6]:
def gradientDescentStep(input, output, weightVector, bValue, learningRate = 0.5):
    # Funcional para parâmetros com formatos: x = (nx, m), y = (1, m), w = (nx, 1), b = (1, 1)
    J = 0; db = 0; m = output.size; nx = weightVector.size; dw = np.zeros((nx, 1))

    for i in range(m): # Calcular para cada amostra (i) do dataset que contém m amostras
        # Variáveis úteis para cada amostra (i)
        xi = input[:,i]                             # Entrada da amostra (i)
        yi = output[0][i]                           # Saída da amostra (i)

        # Cálculo da estimativa realizada pela regressão logística
        zi = np.dot(weightVector.T, xi) + bValue    # z calculado para a amostra (i)
        ai = sigmoid(zi)                            # Saída estimada, ou a da amostra (i)

        # Calculo do erro da amostra (i) é adicionado ao contador para gerar a função de custo (o quão bem w e b estão ajustados com base em TODAS as amostras)
        J += lostFunction(yi, ai)

        # Cálculo das derivadas
        dz = ai - yi
        for d in range(nx):                         # Para cada um dos parâmetros w, calcula-se a derivada dele e adiciona-se ao contador
            dw[d] += xi[d] * dz                     # xi*dz é o valor de dw(1, 2, ..., nx) da amostra (i)
        db += dz                                    # dz é o valor de db da amostra (i)

    # Cálculo dos valores totais (média de todas as amostras)

    J  /= m                                         # Calculo da função de custo, o quão bem w e b estão ajustados para TODAS as amostras
    dw /= m                                         # Calculo de cada uma das derivadas de w, para se dar um passo em direção ao mínimo global (melhor ajuste)
    db /= m                                         # Calculo da derivada do parâmetro b, para se dar um passo em direção ao mínimo global (melhor ajuste)

    # Passo do gradiente descendente para ajustar os parâmetros da IA em direção ao mínimo global
    weightVector -= learningRate * dw
    bValue -= learningRate * db

    return J

In [7]:
def gradientDescentStepVectorized(input, output, weightVector, bValue, learningRate = 0.5):
    # Funcional para parâmetros com formatos: x = (nx, m), y = (1, m), w = (nx, 1), b = (1, 1)
    m = output.size
    Z = np.dot(weightVector.T, input) + bValue
    A = sigmoidArray(Z)
    J = np.sum(lostFunctionArray(output, A))/m
    dZ = A - output
    dW = np.dot(input, dZ.T) / m
    dB = np.sum(dZ) / m

    weightVector -= learningRate * dW
    bValue -= learningRate * dB

    return J

In [10]:
import time

steps = int(6e4)
learningRate = 0.6

aux = int(steps/30)
tic = time.time_ns()
for i in range(steps):
    gradientDescentStepVectorized(X, Y, w, b, learningRate)
    if i % aux == 0:
        print(f"{i/steps*100:.0f}%\t- Erro cometido:\t{gradientDescentStepVectorized(X, Y, w, b, learningRate)*100:.3f}%"
              f"\t\t- Precisão medida:\t{accuracyCalc()[1]*100:.1f}%")
    if i > int(steps/2):
        learningRate = 0.25
    if i > int(steps/1.3):
        learningRate = 0.05

toc = time.time_ns()

print(f"\nTempo de execução: {(toc-tic)/1e9:.3f} s")

0%	- Erro cometido:	21.938%		- Precisão medida:	75.0%
3%	- Erro cometido:	62.909%		- Precisão medida:	65.3%
7%	- Erro cometido:	58.047%		- Precisão medida:	65.2%
10%	- Erro cometido:	60.607%		- Precisão medida:	66.7%
13%	- Erro cometido:	57.543%		- Precisão medida:	65.5%
17%	- Erro cometido:	60.444%		- Precisão medida:	66.7%
20%	- Erro cometido:	57.464%		- Precisão medida:	65.5%
23%	- Erro cometido:	60.412%		- Precisão medida:	66.3%
27%	- Erro cometido:	57.437%		- Precisão medida:	65.5%
30%	- Erro cometido:	60.397%		- Precisão medida:	66.3%
33%	- Erro cometido:	57.420%		- Precisão medida:	65.5%
37%	- Erro cometido:	60.385%		- Precisão medida:	66.3%
40%	- Erro cometido:	57.407%		- Precisão medida:	65.5%
43%	- Erro cometido:	60.374%		- Precisão medida:	66.2%
47%	- Erro cometido:	57.396%		- Precisão medida:	65.5%
50%	- Erro cometido:	60.365%		- Precisão medida:	66.2%
53%	- Erro cometido:	24.436%		- Precisão medida:	74.7%
57%	- Erro cometido:	22.704%		- Precisão medida:	76.0%
60%	- Erro co

In [9]:
print(f"Precisão da IA: {accuracyCalc()[1]*100:.1f}%\n")
print(f"w:\t\t\tb: {b}\n{w}")

Precisão da IA: 75.0%

w:			b: [[4.85005092]]
[[ 7.78291095e-01]
 [ 1.52020536e-01]
 [-1.09411550e+00]
 [ 7.07642000e-02]
 [-6.80384732e-01]
 [-9.57343333e-02]
 [-9.23254114e-01]
 [ 1.25757799e-01]
 [-6.96051519e-01]
 [ 2.01042370e-01]
 [-1.56262128e+00]
 [ 4.78559199e-01]
 [-1.30230002e+00]
 [ 5.25027539e-01]
 [-1.14769668e-01]
 [-7.58319009e-01]
 [-3.80390937e-01]
 [-9.69258816e-01]
 [-3.51461454e-01]
 [-1.93306656e-01]
 [-3.98226409e-01]
 [ 1.62745104e-01]
 [ 8.43587323e-01]
 [ 1.84857188e-01]
 [ 3.06981560e-01]
 [-8.47668984e-01]
 [ 1.30842889e-01]
 [-1.79641325e-03]
 [ 2.33600999e-01]
 [ 1.12134645e+00]
 [-5.74101353e-01]
 [ 4.74233679e-04]
 [-1.74535255e-01]
 [-5.30690199e-01]
 [-1.60431386e-01]
 [-9.49336282e-01]
 [ 7.98484959e-01]
 [-7.67059445e-01]
 [-4.53024115e-01]
 [ 1.46949511e-01]
 [ 7.82132039e-01]
 [ 3.99408826e-01]
 [-1.04320829e+00]
 [-1.56230534e-01]
 [ 1.61059138e-01]
 [ 2.50013156e-01]
 [-1.51777295e-01]
 [-1.11801389e+00]
 [-2.65868877e-01]
 [-5.30667811e-01]
 [-3

In [11]:
print(f"Precisão da IA: {accuracyCalc()[1]*100:.1f}%\n")
print(f"w:\t\t\tb: {b}\n{w}")

Precisão da IA: 75.0%

w:			b: [[5.83849371]]
[[ 0.77194976]
 [ 0.1506866 ]
 [-1.09976433]
 [ 0.05835352]
 [-0.69477483]
 [-0.11400431]
 [-0.94244477]
 [ 0.11736175]
 [-0.70913038]
 [ 0.19773582]
 [-1.57835937]
 [ 0.46527052]
 [-1.31945808]
 [ 0.51577631]
 [-0.1198114 ]
 [-0.77661383]
 [-0.39518645]
 [-0.98007789]
 [-0.36687732]
 [-0.20311653]
 [-0.41302628]
 [ 0.15404697]
 [ 0.84526432]
 [ 0.1842165 ]
 [ 0.29155583]
 [-0.85238826]
 [ 0.11176619]
 [-0.00768103]
 [ 0.22992176]
 [ 1.11646298]
 [-0.58273188]
 [-0.00983843]
 [-0.18605715]
 [-0.53426968]
 [-0.17952407]
 [-0.96305473]
 [ 0.77256803]
 [-0.77573155]
 [-0.47045027]
 [ 0.13783302]
 [ 0.76762855]
 [ 0.37996003]
 [-1.04663539]
 [-0.16643575]
 [ 0.15913886]
 [ 0.23289993]
 [-0.16408245]
 [-1.12222097]
 [-0.28733666]
 [-0.5239992 ]
 [-0.34752518]
 [ 0.40779995]
 [ 0.19298278]
 [-0.85564686]
 [ 0.8256765 ]
 [-0.4382954 ]
 [-0.24044299]
 [ 0.36987032]
 [ 0.16079707]
 [ 1.32865377]
 [ 0.5831952 ]
 [ 1.0913743 ]
 [-0.08884485]
 [ 0.4450

In [13]:
import time

stepNumber = 200

# Versão não vetorizada
tic = time.time_ns()
for i in range(stepNumber):
    gradientDescentStep(X, Y, w, b, 0.01)
toc = time.time_ns()
normalGradientTime = (toc - tic)/1e6 # tempo em milissegundos

# Versão vetorizada
tic = time.time_ns()
for i in range(stepNumber):
    gradientDescentStepVectorized(X, Y, w, b, 0.01)
toc = time.time_ns()
vectorizedGradientTime = (toc - tic)/1e6 # tempo em milissegundos

print (f"\n\nTempo de execução não vetorizado    : {normalGradientTime:.5f} ms\n"
       f"Tempo de execução codigo vetorizado : {vectorizedGradientTime:.5f} ms\n\n"
       f"O código vetorizado é {normalGradientTime/vectorizedGradientTime:.1f} vezes mais rápido em relação ao não vetorizado")



Tempo de execução não vetorizado    : 45209.99900 ms
Tempo de execução codigo vetorizado : 48.00030 ms

O código vetorizado é 941.9 vezes mais rápido em relação ao não vetorizado


In [14]:
print(f"Precisão da IA: {accuracyCalc()[1]*100:.2f}%\n")

print(f"\nComparação de valores:"
      f"\n\ncalc\t|\treal")
prediction = accuracyCalc()[0]
for i in range(m):
    print(f"\n{prediction[0][i]:.2f}\t|\t{Y[0][i]:.2f}\t", end="")
    printed = False
    if (prediction[0][i] > 0.5) and (Y[0][i] > 0.5) and not printed:
        print("(Ok)")
        printed = True
    if (prediction[0][i] <= 0.5) and (Y[0][i] <= 0.5) and not printed:
        print("(Ok)")
        printed = True
    if not printed:
        print("(X)")


Precisão da IA: 75.00%


Comparação de valores:

calc	|	real

0.54	|	0.00	(X)

0.63	|	1.00	(Ok)

0.17	|	1.00	(X)

0.47	|	0.00	(Ok)

0.89	|	0.00	(X)

0.99	|	1.00	(Ok)

0.89	|	1.00	(Ok)

0.24	|	1.00	(X)

0.91	|	0.00	(X)

0.75	|	0.00	(X)

0.08	|	1.00	(X)

0.74	|	1.00	(Ok)

0.64	|	0.00	(X)

0.98	|	1.00	(Ok)

0.04	|	0.00	(Ok)

0.45	|	0.00	(Ok)

0.47	|	0.00	(Ok)

0.83	|	1.00	(Ok)

0.66	|	0.00	(X)

0.13	|	0.00	(Ok)

0.47	|	0.00	(Ok)

0.64	|	1.00	(Ok)

0.32	|	1.00	(X)

0.39	|	1.00	(X)

0.01	|	0.00	(Ok)

0.56	|	0.00	(X)

0.83	|	1.00	(Ok)

0.51	|	0.00	(X)

0.11	|	0.00	(Ok)

0.04	|	0.00	(Ok)

0.05	|	0.00	(Ok)

0.21	|	1.00	(X)

0.10	|	0.00	(Ok)

0.90	|	1.00	(Ok)

0.48	|	1.00	(X)

0.89	|	0.00	(X)

0.46	|	1.00	(X)

0.00	|	0.00	(Ok)

0.70	|	1.00	(Ok)

0.69	|	0.00	(X)

0.17	|	1.00	(X)

0.00	|	0.00	(Ok)

0.31	|	0.00	(Ok)

0.07	|	0.00	(Ok)

0.98	|	1.00	(Ok)

0.61	|	1.00	(Ok)

0.69	|	1.00	(Ok)

0.83	|	1.00	(Ok)

0.28	|	0.00	(Ok)

0.31	|	1.00	(X)

0.02	|	0.00	(Ok)

0.83	|	1.00	(Ok)

0.93	|	1.00	(Ok)

0.69